In [ ]:
import torch
import torch.nn as nn

# Define the LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)
    
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.fc(out[:, -1, :])  # Use the last output of the sequence
        return out

# Define the dimensions
input_size = 2  # Number of features (electrodes)
sequence_length = 2000  # Length of the sequence
hidden_size = 64  # Hidden size of the LSTM
num_classes = 2  # Number of output classes

# Reshape the data
X_train_reshaped = basal_windows.reshape((-1, sequence_length, input_size))

# Convert to PyTorch tensors
tensor_X_train = torch.Tensor(X_train_reshaped)
tensor_y_train = torch.Tensor(y_train)

# Create Tensor datasets
train_data = torch.utils.data.TensorDataset(tensor_X_train, tensor_y_train)

# Define the batch size
batch_size = 16

# Create the data loader
train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)

# Initialize the LSTM model
model = LSTMModel(input_size, hidden_size, num_classes)

# Move the model to the GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


num_epochs = 100
# Training loop
for epoch in range(num_epochs):
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device).long()  # Convert labels to LongTensor

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Print the loss
        if (i+1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

